# Импорт библиотек

In [1]:
import torch
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer, AutoTokenizer, AutoModelForSequenceClassification, pipeline

/Users/denissamatov/Studies/Фин. Мат./Курсовая работа/news-impact-on-stock-prices/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Тестовые данные

In [3]:
# Данные для анализа
financial_text_russian = [
    "Компания X объявила о рекордных прибылях за последний квартал.",
    "Компания X объявила о рекордных убытках за последний квартал.",
    "Компания X имеет показатели без изменений.",
    "Электромобильный стартап Arrival экс-главы Yota уйдет из России.",
    "Шрёдер отклонил предложение войти в совет директоров «Газпрома»",
    "Шельф берут в разработку // Генподрядчиком «Газпрома» на море может стать компания Андрея Патрушева",
    "Чистая прибыль 'РусГидро' по РСБУ за 1 полугодие выросла на 17%",
    "Финский производитель шин Nokian Tyres решил уйти из России",
    "Федун ушел с поста вице-президента ЛУКОЙЛа на пенсию",
    "Промсвязьбанк укрепил свои позиции в топ-10 российских банков по портфелю кредитов организациям."
]

# Тестирование моделей

In [4]:
# Функция перевода текста с русского на английский
def translate_text(texts, model_name="Helsinki-NLP/opus-mt-ru-en"):
    translation_model = MarianMTModel.from_pretrained(model_name)
    translation_tokenizer = MarianTokenizer.from_pretrained(model_name)
    inputs = translation_tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    outputs = translation_model.generate(**inputs)
    return translation_tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Функция для классификации на английском языке с использованием FinBERT
def classify_with_finbert(texts):
    tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
    model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return predictions

# Функция для классификации на русском языке с использованием RuBERT
def classify_with_rubert(texts, model_name='blanchefort/rubert-base-cased-sentiment'):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    return predictions

In [8]:
# Переводим текст на английский
translated_text = translate_text(financial_text_russian)
print("Translated Text:", translated_text)

# Классификация с FinBERT
finbert_predictions = classify_with_finbert(translated_text)

# Обработка результатов FinBERT
finbert_positive = finbert_predictions[:, 0].tolist()
finbert_negative = finbert_predictions[:, 1].tolist()
finbert_neutral = finbert_predictions[:, 2].tolist()
finbert_table = {
    'Headline': financial_text_russian,
    'En Headline': translated_text,
    "Positive": finbert_positive,
    "Negative": finbert_negative,
    "Neutral": finbert_neutral
}
finbert_df = pd.DataFrame(finbert_table)
finbert_df['real_score'] = finbert_df['Positive'] - finbert_df['Negative']
print("\nFinBERT Analysis:")
finbert_df

/Users/denissamatov/Studies/Фин. Мат./Курсовая работа/news-impact-on-stock-prices/venv/lib/python3.12/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Translated Text: ['Company X has announced record profits for the last quarter.', 'Company X announced record losses over the last quarter.', 'Company X has no change.', 'The electric motor startup of the Arrival ex-head Yota will leave Russia.', 'Schroeder declined an offer to join the Gazprom Board of Directors.', "Shelf is being developed / Gazproma's contractor at sea may be Andrei Patrushev's company.", 'The net profit of RusHidro on ASHI increased by 17% in the first half of the year', 'The Finnish tyre manufacturer Nokian Tyres decided to leave Russia.', 'Fedun retired as Vice President of Lucoil.', 'Promsbank has strengthened its position in the top 10 of Russian banks on the loan portfolio to organizations.']

FinBERT Analysis:


,Headline,En Headline,Positive,Negative,Neutral,real_score
0,Компания X объявила о рекордных прибылях за по...,Company X has announced record profits for the...,0.948517,0.025460,0.026023,0.923057
1,Компания X объявила о рекордных убытках за пос...,Company X announced record losses over the las...,0.009817,0.972690,0.017493,-0.962874
2,Компания X имеет показатели без изменений.,Company X has no change.,0.024528,0.030658,0.944815,-0.006130
3,Электромобильный стартап Arrival экс-главы Yot...,The electric motor startup of the Arrival ex-h...,0.035713,0.033495,0.930791,0.002218
4,Шрёдер отклонил предложение войти в совет дире...,Schroeder declined an offer to join the Gazpro...,0.043275,0.024247,0.932478,0.019028
5,Шельф берут в разработку // Генподрядчиком «Га...,Shelf is being developed / Gazproma's contract...,0.033618,0.024472,0.941910,0.009146
6,Чистая прибыль 'РусГидро' по РСБУ за 1 полугод...,The net profit of RusHidro on ASHI increased b...,0.954949,0.014268,0.030784,0.940681
7,Финский производитель шин Nokian Tyres решил у...,The Finnish tyre manufacturer Nokian Tyres dec...,0.016718,0.811226,0.172056,-0.794508
8,Федун ушел с поста вице-президента ЛУКОЙЛа на ...,Fedun retired as Vice President of Lucoil.,0.019642,0.112515,0.867843,-0.092873
9,Промсвязьбанк укрепил свои позиции в топ-10 ро...,Promsbank has strengthened its position in the...,0.959419,0.017457,0.023124,0.941961


In [9]:
# Классификация с RuBERT
rubert_predictions = classify_with_rubert(financial_text_russian)

# Обработка результатов RuBERT
rubert_positive = rubert_predictions[:, 0].tolist()
rubert_negative = rubert_predictions[:, 1].tolist()
rubert_neutral = rubert_predictions[:, 2].tolist()
rubert_table = {
    'Headline': financial_text_russian,
    "Positive": rubert_positive,
    "Negative": rubert_negative,
    "Neutral": rubert_neutral
}
rubert_df = pd.DataFrame(rubert_table)
print("\nRuBERT Analysis:")
rubert_df

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



RuBERT Analysis:


,Headline,Positive,Negative,Neutral
0,Компания X объявила о рекордных прибылях за по...,0.806638,0.160117,0.033245
1,Компания X объявила о рекордных убытках за пос...,0.804880,0.159613,0.035507
2,Компания X имеет показатели без изменений.,0.806738,0.152314,0.040948
3,Электромобильный стартап Arrival экс-главы Yot...,0.816962,0.151060,0.031978
4,Шрёдер отклонил предложение войти в совет дире...,0.180748,0.067817,0.751434
5,Шельф берут в разработку // Генподрядчиком «Га...,0.182503,0.068359,0.749138
6,Чистая прибыль 'РусГидро' по РСБУ за 1 полугод...,0.820529,0.147761,0.031711
7,Финский производитель шин Nokian Tyres решил у...,0.814953,0.140481,0.044566
8,Федун ушел с поста вице-президента ЛУКОЙЛа на ...,0.802257,0.136790,0.060953
9,Промсвязьбанк укрепил свои позиции в топ-10 ро...,0.819028,0.149560,0.031412


In [7]:
# Дополнительная классификация с другой моделью через pipeline (опционально)
def classify_with_pipeline(texts, model_name="seara/rubert-tiny2-russian-sentiment"):
    classifier = pipeline("text-classification", model=model_name)
    return classifier(texts)

pipeline_results = classify_with_pipeline(financial_text_russian)
print("\nPipeline Analysis:")
for result in pipeline_results:
    print(result)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.



Pipeline Analysis:
{'label': 'neutral', 'score': 0.5374321937561035}
{'label': 'neutral', 'score': 0.5123931765556335}
{'label': 'neutral', 'score': 0.7227913737297058}
{'label': 'neutral', 'score': 0.5539339780807495}
{'label': 'neutral', 'score': 0.5862728953361511}
{'label': 'neutral', 'score': 0.9074979424476624}
{'label': 'neutral', 'score': 0.5426933765411377}
{'label': 'neutral', 'score': 0.8176657557487488}
{'label': 'neutral', 'score': 0.6004788875579834}
{'label': 'neutral', 'score': 0.6724496483802795}
